In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import folium
from branca.element import Template, MacroElement

In [2]:
external_dir = "../../data/external"
interim_dir = "../../data/interim"
processed_dir = "../../data/processed"

map_file_path = "../../reports/maps/respondent_segments_map_with_layers_and_legend.html"

##### We want to make a map of origin/destination trip coordinates for arriving/departing passengers. The segmentation required is by resident and visitor, also by transit

In [3]:
clean_data_model_file = os.path.join(processed_dir, "atc_travel_survey_final_data.csv")
clean_data_model_df = pd.read_csv(clean_data_model_file)

C:\Users\mwe\AppData\Local\Temp\ipykernel_24348\911404828.py:2: DtypeWarning: Columns (1,10,21,23,24,30,31,45,52,54,56,57,60,78,85,86,89,95,101,103,114,115,116,119,129,132,135,139,155,156,161,170,172,173,175,233) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_data_model_df = pd.read_csv(clean_data_model_file)


In [4]:
clean_data_model_df.head()

,unique_id,respondentid,date_completed,time_completed,is_pilot,is_self_administered,record_type_synthetic,access_mode_frequency,access_mode_frequency_label,access_mode,...,trip_arrival_time,trip_arrival_time_label,trip_start_time,trip_start_time_label,model_respondent_bool,weight_departing_and_arriving,weight_departing_only,weight_departing_only_model_respondents,weight_non_sas_departing_only,weight_departing_only_with_time_of_day
0,1,5473,2024-10-04,08:41:12,False,False,0,NaN,NaN,NaN,...,1.0,FIVE_TO_FIVE_THIRTY,1.0,FIVE_TO_FIVE_THIRTY,True,10.840259,11.525574,11.622594,19.179428,10.874504
1,2,5476,2024-10-04,08:40:04,False,False,0,NaN,NaN,1.0,...,10.0,NINE_THIRTY_TO_TEN,8.0,EIGHT_THIRTY_TO_NINE,True,6.487856,3.355533,3.346428,8.687559,6.502862
2,3,5489,2024-10-04,08:51:36,False,False,0,NaN,NaN,NaN,...,3.0,SIX_TO_SIX_THIRTY,3.0,SIX_TO_SIX_THIRTY,True,10.840259,11.525574,11.622594,19.179428,10.874504
3,4,5558,2024-10-04,10:32:58,False,False,0,NaN,NaN,1.0,...,14.0,ELEVEN_THIRTY_TO_NOON,13.0,ELEVEN_TO_ELEVEN_THIRTY,True,6.487856,3.355533,3.346428,8.687559,6.502862
4,5,5593,2024-10-04,11:09:46,False,False,0,NaN,NaN,1.0,...,15.0,NOON_TO_TWELVE_THIRTY,15.0,NOON_TO_TWELVE_THIRTY,True,6.487856,3.355533,3.346428,8.687559,6.502862


In [5]:
working_df = clean_data_model_df[['unique_id', 'is_pilot', 'is_self_administered', 'record_type_synthetic', 'respondentid', 'marketsegment_label', 'passenger_segment_label', 'main_mode_label', 'origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude']]

In [6]:
working_df = working_df[working_df['record_type_synthetic']==0]

In [7]:
working_df.shape

(4971, 12)

In [8]:
working_df.head()

,unique_id,is_pilot,is_self_administered,record_type_synthetic,respondentid,marketsegment_label,passenger_segment_label,main_mode_label,origin_latitude,origin_longitude,destination_latitude,destination_longitude
0,1,False,False,0,5473,EMPLOYEE,NaN,PERSONAL_CAR_PARKED,32.816714,-117.176898,32.732627,-117.204064
1,2,False,False,0,5476,EMPLOYEE,NaN,BUS_992,32.639943,-117.085774,32.733783,-117.193315
2,3,False,False,0,5489,EMPLOYEE,NaN,PERSONAL_CAR_PARKED,32.743009,-117.131699,32.732627,-117.204064
3,4,False,False,0,5558,EMPLOYEE,NaN,BUS_992,32.706752,-117.148810,32.732627,-117.204064
4,5,False,False,0,5593,EMPLOYEE,NaN,BUS_992,32.678108,-117.099196,32.732627,-117.204064


In [9]:
working_df['main_mode_label'].value_counts()

main_mode_label
RIDEHAIL_TAXI                         1559
PERSONAL_CAR_DROPPED_OFF_PICKED_UP    1487
PERSONAL_CAR_PARKED                    645
RENTAL_CAR                             625
AIRPORT_FLYER_SHUTTLE                  232
BUS_992                                188
SHARED_SHUTTLE_VAN                     149
OTHER                                   35
PUBLIC_TRANSPORTATION                   26
WALK                                    14
MICROMOBILITY_PERSONAL                  10
MICROMOBILITY_SHARED                     1
Name: count, dtype: int64

In [10]:
# Define is_transit_user
working_df['is_transit_user'] = np.where(
    working_df['main_mode_label'].isin(['BUS_992', 'AIRPORT_FLYER_SHUTTLE', 'PUBLIC_TRANSPORTATION']),
    True,
    False
)

# Define respondent_segment
working_df['respondent_segment'] = np.where(
    working_df['marketsegment_label'] == 'EMPLOYEE', 'Employee',
    np.where(
        working_df['passenger_segment_label'].str.startswith('RESIDENT_'), 'Resident',
        np.where(
            working_df['passenger_segment_label'].str.startswith('VISITOR_'), 'Visitor',
            'Unknown'
        )
    )
)

# Define latitude
working_df['latitude'] = np.where(
    (working_df['respondent_segment'] == 'Employee') |
    (working_df['passenger_segment_label'].str.endswith('DEPARTING', na=False)),
    working_df['origin_latitude'],
    np.where(
        working_df['passenger_segment_label'].str.endswith('ARRIVING', na=False),
        working_df['destination_latitude'],
        np.nan
    )
)

# Define longitude
working_df['longitude'] = np.where(
    (working_df['respondent_segment'] == 'Employee') |
    (working_df['passenger_segment_label'].str.endswith('DEPARTING', na=False)),
    working_df['origin_longitude'],
    np.where(
        working_df['passenger_segment_label'].str.endswith('ARRIVING', na=False),
        working_df['destination_longitude'],
        np.nan
    )
)

In [11]:
working_df.head()

,unique_id,is_pilot,is_self_administered,record_type_synthetic,respondentid,marketsegment_label,passenger_segment_label,main_mode_label,origin_latitude,origin_longitude,destination_latitude,destination_longitude,is_transit_user,respondent_segment,latitude,longitude
0,1,False,False,0,5473,EMPLOYEE,NaN,PERSONAL_CAR_PARKED,32.816714,-117.176898,32.732627,-117.204064,False,Employee,32.816714,-117.176898
1,2,False,False,0,5476,EMPLOYEE,NaN,BUS_992,32.639943,-117.085774,32.733783,-117.193315,True,Employee,32.639943,-117.085774
2,3,False,False,0,5489,EMPLOYEE,NaN,PERSONAL_CAR_PARKED,32.743009,-117.131699,32.732627,-117.204064,False,Employee,32.743009,-117.131699
3,4,False,False,0,5558,EMPLOYEE,NaN,BUS_992,32.706752,-117.148810,32.732627,-117.204064,True,Employee,32.706752,-117.148810
4,5,False,False,0,5593,EMPLOYEE,NaN,BUS_992,32.678108,-117.099196,32.732627,-117.204064,True,Employee,32.678108,-117.099196


In [12]:
working_df['respondent_segment'].value_counts()

respondent_segment
Visitor     2549
Resident    1763
Employee     659
Name: count, dtype: int64

In [13]:
working_df['is_transit_user'].value_counts()

is_transit_user
False    4525
True      446
Name: count, dtype: int64

In [14]:
# Create base map
m = folium.Map(location=[32.73, -117.16], zoom_start=11)

# Segment names and colors
segment_labels = {
    'Resident - Transit': 'blue',
    'Resident - Non-Transit': 'darkblue',
    'Visitor - Transit': 'green',
    'Visitor - Non-Transit': 'darkgreen',
    'Employee - Transit': 'orange',
    'Employee - Non-Transit': 'red'
}

# Create a FeatureGroup for each segment
layers = {label: folium.FeatureGroup(name=label, show=True) for label in segment_labels}

# Add markers to their respective layer
for _, row in working_df.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        segment_key = f"{row['respondent_segment']} - {'Transit' if row['is_transit_user'] else 'Non-Transit'}"
        color = segment_labels.get(segment_key, 'gray')

        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_opacity=0.6,
            popup=segment_key
        ).add_to(layers[segment_key])

# Add layers to map
for fg in layers.values():
    fg.add_to(m)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Add custom legend
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 220px; height: 190px;
    z-index:9999; font-size:14px;
    background-color: white;
    padding: 10px;
    border:2px solid grey;
    border-radius:8px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
">
<b>Legend</b><br>
<span style="color:blue;">●</span> Resident - Transit<br>
<span style="color:darkblue;">●</span> Resident - Non-Transit<br>
<span style="color:green;">●</span> Visitor - Transit<br>
<span style="color:darkgreen;">●</span> Visitor - Non-Transit<br>
<span style="color:orange;">●</span> Employee - Transit<br>
<span style="color:red;">●</span> Employee - Non-Transit<br>
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# Save map
m.save(map_file_path)
